In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('wordnet')
import gensim
import string
from wordcloud import WordCloud
from PIL import Image
import collections

[nltk_data] Downloading package wordnet to /home/quadm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [57]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

def hasNumber(word):
    return any(char.isdigit() for char in word)
      
def preprocess(textfile):
    with open(textfile, 'r') as reader:
        text = reader.read()
    text = text.lower()     # Make all text lowercase
    nopunct="".join([char for char in text if char not in string.punctuation]) # Remove all punctuation
    tokens = nltk.tokenize.word_tokenize(nopunct) # Tokenize
    nonums = [t for t in tokens if not hasNumber(t)]  # Remove all numbers
    
    ps = PorterStemmer()
    
    nostop = [t for t in nonums if t not in nltk.corpus.stopwords.words('english')]
    return [ps.stem(t) for t in nostop]

tokenized = preprocess('data/C50train/AlanCrosby/10306newsML.txt')

In [64]:
import os
from tqdm  import tqdm

train_dir = 'data/C50train/'
train_corpus = []
for author in tqdm(os.listdir(train_dir)):
    for sample_text in os.listdir(os.path.join(train_dir, author)):
        train_corpus.append(preprocess(os.path.join(train_dir, author, sample_text)))

100%|██████████| 50/50 [01:38<00:00,  1.97s/it]


In [68]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(sentences=train_corpus, size=100, window=5, min_count=1, workers=4)
model.train([tokenized], total_examples=2500, epochs=100)

(49549, 51900)

In [73]:
model.save('articles.mode')

KeyError: "word 'why' not in vocabulary"